In [3]:
import yahooquery
import requests
import json
from yahooquery import Ticker
import time
import json
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [16]:
class StockDetail():

    api_key = 'ADLK0ZK57SB9BJKX'

    def __init__(self, symbol):

        self.symbol = symbol
    
    #returns earnings trend data
    def get_earnings_trend(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.earnings_trend

        output_dict = {}
        output_dict['currentQtr'] = raw_dict[self.symbol]['trend'][0]['epsTrend']
        output_dict['nextQtr'] = raw_dict[self.symbol]['trend'][1]['epsTrend']
        output_dict['currentYr'] = raw_dict[self.symbol]['trend'][2]['epsTrend']
        output_dict['nextYr'] = raw_dict[self.symbol]['trend'][3]['epsTrend']

        return output_dict


### WORKS USING ALPHA VANTAGE

In [17]:
global AlphaVantageKey
#basic key AlphaVantageKey = 'XEQ9AFU8KM035KMG'
#premium api key
AlphaVantageKey = 'ZTB6U564ILR50HU3'
# @cache_request()
def fairValue_hist(ticker):
    global AlphaVantageKey

    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
    r = requests.get(url)
    data = r.json()
    # print(type(data['annualEarnings']))
    # display(data['fannualEarnings'])
    df = pd.DataFrame(data['annualEarnings'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retriving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='7y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
        display(priceHistory)
#         priceHistory['date'] = priceHistory['date'].dt.strftime('%Y-%m-%d')
        print('dtypes')
        display(priceHistory.dtypes)
        priceHistory.drop(priceHistory.tail(1).index,inplace=True)
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
#         print('testdf')
#         display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

    print('fv_df_currentYR: ', fv_df)
    return fv_df


fairValue_hist('aapl')

,symbol,date,open,high,low,close,volume,adjclose,dividends,splits
0,aapl,2016-03-09,25.327499,25.395000,25.067499,25.280001,108806800,23.233486,0.0,0.0
1,aapl,2016-03-10,25.352501,25.559999,25.037500,25.292500,134054400,23.244974,0.0,0.0
2,aapl,2016-03-11,25.559999,25.570000,25.375000,25.565001,109632800,23.495413,0.0,0.0
3,aapl,2016-03-14,25.477501,25.727501,25.445000,25.629999,100304400,23.555155,0.0,0.0
4,aapl,2016-03-15,25.990000,26.295000,25.962500,26.145000,160270800,24.028458,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1757,aapl,2023-03-02,144.380005,146.710007,143.899994,145.910004,52238100,145.910004,0.0,0.0
1758,aapl,2023-03-03,148.039993,151.110001,147.330002,151.029999,70668500,151.029999,0.0,0.0
1759,aapl,2023-03-06,153.789993,156.300003,153.460007,153.830002,87558000,153.830002,0.0,0.0
1760,aapl,2023-03-07,153.699997,154.029999,151.130005,151.600006,56182000,151.600006,0.0,0.0


dtypes


symbol        object
date          object
open         float64
high         float64
low          float64
close        float64
volume         int64
adjclose     float64
dividends    float64
splits       float64
dtype: object

,symbol,date,open,high,low,close,volume,adjclose,dividends,splits
0,aapl,2016-03-09,25.327499,25.395000,25.067499,25.280001,108806800,23.233486,0.0,0.0
1,aapl,2016-03-10,25.352501,25.559999,25.037500,25.292500,134054400,23.244974,0.0,0.0
2,aapl,2016-03-11,25.559999,25.570000,25.375000,25.565001,109632800,23.495413,0.0,0.0
3,aapl,2016-03-14,25.477501,25.727501,25.445000,25.629999,100304400,23.555155,0.0,0.0
4,aapl,2016-03-15,25.990000,26.295000,25.962500,26.145000,160270800,24.028458,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1757,aapl,2023-03-02,144.380005,146.710007,143.899994,145.910004,52238100,145.910004,0.0,0.0
1758,aapl,2023-03-03,148.039993,151.110001,147.330002,151.029999,70668500,151.029999,0.0,0.0
1759,aapl,2023-03-06,153.789993,156.300003,153.460007,153.830002,87558000,153.830002,0.0,0.0
1760,aapl,2023-03-07,153.699997,154.029999,151.130005,151.600006,56182000,151.600006,0.0,0.0


dtypes


...

,symbol,date,open,high,low,close,volume,adjclose,dividends,splits
0,aapl,2016-03-09,25.327499,25.395000,25.067499,25.280001,108806800,23.233486,0.0,0.0
1,aapl,2016-03-10,25.352501,25.559999,25.037500,25.292500,134054400,23.244974,0.0,0.0
2,aapl,2016-03-11,25.559999,25.570000,25.375000,25.565001,109632800,23.495413,0.0,0.0
3,aapl,2016-03-14,25.477501,25.727501,25.445000,25.629999,100304400,23.555155,0.0,0.0
4,aapl,2016-03-15,25.990000,26.295000,25.962500,26.145000,160270800,24.028458,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1757,aapl,2023-03-02,144.380005,146.710007,143.899994,145.910004,52238100,145.910004,0.0,0.0
1758,aapl,2023-03-03,148.039993,151.110001,147.330002,151.029999,70668500,151.029999,0.0,0.0
1759,aapl,2023-03-06,153.789993,156.300003,153.460007,153.830002,87558000,153.830002,0.0,0.0
1760,aapl,2023-03-07,153.699997,154.029999,151.130005,151.600006,56182000,151.600006,0.0,0.0


dtypes


symbol        object
date          object
open         float64
high         float64
low          float64
close        float64
volume         int64
adjclose     float64
dividends    float64
splits       float64
dtype: object

fv_df_currentYR:     reportedEPS2019  reportedEPS2018  reportedEPS2017  avgPrice2019  \
0             2.98             2.97              2.3     77.631375   

   avgPrice2018  avgPrice2017    p_e2019    p_e2018    p_e2017  \
0      38.29425     43.680395  26.050797  12.893687  18.991476   

   currentYrEstimate  FairValue  
0               5.97  115.29256  


,reportedEPS2019,reportedEPS2018,reportedEPS2017,avgPrice2019,avgPrice2018,avgPrice2017,p_e2019,p_e2018,p_e2017,currentYrEstimate,FairValue
0,2.98,2.97,2.3,77.631375,38.29425,43.680395,26.050797,12.893687,18.991476,5.97,115.29256
